<a href="https://colab.research.google.com/github/roxyrong/w266_project/blob/main/t5_finetune_text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install sentencepiece
!pip install accelerate -U
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
%cd /content/drive/MyDrive/Github/w266_project

import sys
project_path = '/content/drive/MyDrive/Github/w266_project'
sys.path.append(project_path)

In [ ]:
import os
from typing import Dict, List
import subprocess
import collections
import json
import random
import numpy as np
import pandas as pd
import nltk
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
# for evaluation
nltk.download('punkt')

In [ ]:
# from third_party.spider.process_sql import Schema, get_sql, get_schema

In [ ]:
# datasets
with open('spider/train_spider.json', 'r') as f:
    train_spider = pd.read_json(f)
with open('spider/train_others.json', 'r') as f:
    others_spider = pd.read_json(f)
with open('spider/dev.json', 'r') as f:
    dev_spider = pd.read_json(f)

In [ ]:
# load schema for all tables
with open('spider/tables.json', 'r') as f:
    schema_df = pd.read_json(f)

# db_dir = f"spider/database"
# schema_dict = {}
# column_names_dict = {}
# for idx, row in schema_df.iterrows():
#     db = row['db_id']
#     db_path = os.path.join(db_dir, db, db + ".sqlite")
#     schema = str(get_schema(db_path))
#     schema_dict[db] = schema
#     column_name = json.loads(schema.replace("'", "\""))
#     column_names_dict[db]= {"table_id": list(column_name.keys()), "column_name": schema}

In [ ]:
def _get_schema_string(table_json):
  """Returns the schema serialized as a string."""
  table_id_to_column_names = collections.defaultdict(list)
  for table_id, name in table_json["column_names_original"]:
    table_id_to_column_names[table_id].append(name.lower())
  tables = table_json["table_names_original"]

  table_strings = []
  for table_id, table_name in enumerate(tables):
    column_names = table_id_to_column_names[table_id]
    table_string = " | %s : %s" % (table_name.lower(), " , ".join(column_names))
    table_strings.append(table_string)

  return "".join(table_strings)

schema_dict = {}
for idx, row in schema_df.iterrows():
  db_id = row['db_id']
  schema = _get_schema_string(row)
  schema_dict[db_id] = schema

In [ ]:
# shuffle the dataset

train_spider = train_spider.iloc[np.random.permutation(train_spider.index)].reset_index(drop=True)
others_spider = train_spider.iloc[np.random.permutation(others_spider.index)].reset_index(drop=True)

# Model Architecture

In [ ]:
# construct prompt

prefix = 'translate English to SQL:'

train_spider['schema'] = train_spider['db_id'].map(schema_dict)
train_spider['prompt'] = prefix + train_spider['question'] + '\nDatabse schema is ' + train_spider['schema']
others_spider['schema'] = others_spider['db_id'].map(schema_dict)
others_spider['prompt'] = prefix + others_spider['question'] + '\nDatabse schema is ' + others_spider['schema']
dev_spider['schema'] = dev_spider['db_id'].map(schema_dict)
dev_spider['prompt'] = prefix + dev_spider['question'] + '\nDatabse schema is ' + dev_spider['schema']

In [ ]:
def preprocess_data(text_pair, tokenizer, max_length=128):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]

    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    label_ids = target_encoded['input_ids'][0]

    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:

    def __init__(self,
                 df,
                 tokenizer,
                 max_load_at_once,
                 max_length=128,
                 shuffle=True):

        self.df = df
        self.tokenizer = tokenizer
        self.n_examples = len(df)
        self.max_load_at_once = max_load_at_once
        self.max_length = max_length
        self.shuffle = shuffle

        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()

        # Load first chunk of max_load_at_once examples
        self.df_curr_loaded = self._load_next_chunk(0)
        self.curr_idx_in_load = 0

    def _load_next_chunk(self, idx):
        load_start = idx
        load_end = idx + self.max_load_at_once

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this chunk
        self.df_curr_loaded = self.df.iloc[load_start:load_end].sample(frac=1)

    def __len__(self):
        return self.n_examples

    def __getitem__(self, idx):
        if self.df_curr_loaded is None or self.curr_idx_in_load >= len(self.df_curr_loaded):
            self._load_next_chunk(idx)
            self.curr_idx_in_load = 0

        text_pair = self.df_curr_loaded[['prompt', 'query']].values.astype(str)[self.curr_idx_in_load]
        self.curr_idx_in_load += 1

        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data

    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)

            if i == self.__len__()-1:
                self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

# Model Training

In [ ]:
model_name = "t5-base"

folder_name = f"{model_name}-finetuned-spider_1"
train_path = f"{project_path}/{folder_name}"
model_path = train_path + f'/{model_name}-finetuned'
last_check_point = train_path + f'/checkpoint-8500'

In [ ]:
%%capture

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
max_length = 512
max_load_at_once = 100

train_data_iterator = TranslationDataIterator(
    df=train_spider,
    tokenizer=tokenizer,
    max_load_at_once=max_load_at_once,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    df=others_spider,
    tokenizer=tokenizer,
    max_load_at_once=max_load_at_once,
    max_length=max_length
)

In [ ]:
batch_size = 16

args = Seq2SeqTrainingArguments(
    train_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# start from scratch
trainer.train()

# start from a checkpoint
# trainer.train(resume_from_checkpoint= last_check_point)

In [ ]:
trainer.save_model(model_path)

# Evaluation

In [ ]:
finetune_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [ ]:
# evaluate
max_length = 128

inputs = tokenizer.batch_encode_plus(
        list(dev_spider['prompt']),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

output_tokens = finetune_model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128
)

outputs = [tokenizer.decode(i, skip_special_tokens=True) for i in output_tokens]

In [ ]:
with open(f'{folder_name}/predicted_result.txt', 'w') as f:
    for idx, output in enumerate(outputs):
        db_id = dev_spider.iloc[idx]['db_id']
        f.write(output + '\t' + db_id + '\n')

In [ ]:
# evaluate results
eval_path = f"third_party/spider/evaluation.py"
gold = f"third_party/spider/evaluation_examples/gold_example.txt"
pred = f"{folder_name}/predicted_result.txt"
db_dir = f"spider/database"
table = f"spider/tables.json"
etype = "all"

cmd_str = f"python3 \"{eval_path}\" --gold \"{gold}\" --pred \"{pred}\" --db \"{db_dir}\" --table \"{table}\" --etype {etype} "
result = subprocess.run(cmd_str, shell=True, capture_output=True, text=True)

In [ ]:
import pprint
pprint.pprint(result.stdout[-4633:])